In [24]:
# With the data, let's put all in the same date format, merge in a single dataframe and save to a CSV file

In [25]:
import pandas as pd
import glob

raw_path = "C:\\Users\\DELL\\OneDrive\\Área de Trabalho\\Ironhack\\final project\\data\\raw\\"

dfs = {}

for file in glob.glob(raw_path + "*.csv"):
    name = file.split("\\")[-1].replace(".csv", "")
    df = pd.read_csv(file)

    # date to datetime
    df["data"] = pd.to_datetime(df["data"], errors="coerce")

    # rename the value column
    df = df.rename(columns={"valor": name})

    dfs[name] = df

print("Loaded keys:", dfs.keys())


Loaded keys: dict_keys(['debt_dbgg', 'ibc_br', 'ipca', 'selic_efetiva', 'selic_meta', 'unemployment', 'usd_brl'])


In [26]:
dfs["usd_brl_monthly"] = (
    dfs["usd_brl"]
    .set_index("data")
    .resample("MS")
    .mean()
    .reset_index()
)

In [27]:
del dfs["usd_brl"]  # remove the daily dataframe

In [30]:
df = dfs["selic_meta"]

for name, temp in dfs.items():
    if name == "selic_meta":
        continue

    df = df.merge(temp, on="data", how="inner")

df = df.sort_values("data")
df.head()

,data,selic_meta,debt_dbgg,ibc_br,ipca,selic_efetiva,unemployment,usd_brl
0,2015-11-01,14.15,23.60,95.53474,1.01,1.06,9.1,3.849900
1,2015-12-01,14.15,25.03,95.64529,0.96,1.16,9.1,3.870495
2,2016-01-01,14.15,25.18,90.36460,1.27,1.06,9.6,4.051715
3,2016-02-01,14.15,27.00,91.98494,0.90,1.00,10.3,3.973116
4,2016-03-01,14.15,28.09,98.52596,0.43,1.16,11.0,3.703309


In [33]:
# Target variable

df["selic_next"] = df["selic_meta"].shift(-1)
df["target"] = (df["selic_next"] > df["selic_meta"]).astype(int)

In [35]:
df.head()

,data,selic_meta,debt_dbgg,ibc_br,ipca,selic_efetiva,unemployment,usd_brl,selic_next,target
0,2015-11-01,14.15,23.60,95.53474,1.01,1.06,9.1,3.849900,14.15,0
1,2015-12-01,14.15,25.03,95.64529,0.96,1.16,9.1,3.870495,14.15,0
2,2016-01-01,14.15,25.18,90.36460,1.27,1.06,9.6,4.051715,14.15,0
3,2016-02-01,14.15,27.00,91.98494,0.90,1.00,10.3,3.973116,14.15,0
4,2016-03-01,14.15,28.09,98.52596,0.43,1.16,11.0,3.703309,14.15,0


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   data           119 non-null    datetime64[ns]
 1   selic_meta     119 non-null    float64       
 2   debt_dbgg      119 non-null    float64       
 3   ibc_br         119 non-null    float64       
 4   ipca           119 non-null    float64       
 5   selic_efetiva  119 non-null    float64       
 6   unemployment   119 non-null    float64       
 7   usd_brl        119 non-null    float64       
 8   selic_next     118 non-null    float64       
 9   target         119 non-null    int64         
dtypes: datetime64[ns](1), float64(8), int64(1)
memory usage: 9.4 KB


In [39]:
df.to_csv("C:\\Users\\DELL\\OneDrive\\Área de Trabalho\\Ironhack\\final project\\data\\cleaned\\" + "economic_master.csv", index=False)